In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from helper_functions_pipe_testing import *
from sklearn.metrics import  f1_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import *


#Instructions for the pipeline
Requires four inputs:
    - Mass spec data with corresponding NP surface characteristics and experimental conditions (time, concentration)
    - Proteome data from uniprot (Mass, length, sequence, GO for later analysis)
    - NetsurfP data for the proteins that are to be searched
    - X characteristics to predict

pipeline
Take mass spec spreadsheet
Accession,Enrichment,Dh,TEM,Zp,BET,Composition,Ligand,Shape,IncubationTime,IncubationConcentration
Merge with Proteome data to get file that has
Accession,Enrichment,Dh,TEM,Zp,BET,Composition,Ligand,Shape,IncubationTime,IncubationConcentration,Mass,Length,Sequence
Calculate protein features using biopython
Merge with NSP data to get all protein features

Split into X and Y dataset with Entries as labels

In [16]:
### New Data workup for RFG

# Pull together Proteomic data
in_dir="Input_data/Proteomic data/abundance/"
#Mass Spec data input in one excel spreadsheet - Entry - Abundance labeled by NP Unique ID
#Abundance as a percent
files= os.listdir(in_dir)
for i,f in enumerate(files):
    if i==0:
        raw_MS_data=pd.read_excel(in_dir+f,header=0)
    else:
        temp = pd.read_excel(in_dir+f,header=0)
        raw_MS_data=raw_MS_data.merge(temp,how='outer',on='Entry')
# melt to make it an accession number, NPID, Abundance dataset
raw_MS_data = pd.melt(raw_MS_data, id_vars=['Entry'],var_name='NPUNID', value_name='Abundance')
#remove prots that were added due to merge
raw_MS_data=raw_MS_data.dropna()

C:\Users\kmp95\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [42]:
###Bring in controls (MS data for serums)##
controls=pd.read_excel('Input_data/Proteomic data/controls.xlsx',header=0)
MS_data_controls = pd.merge(raw_MS_data,controls,how='inner', on='Entry')
###Bring in Uniprot_data,NSPdata and NP data##
uniprot_filepath='UniProt/Bovine_Mouse_proteome_biopyCalcs.xlsx'
uniprot_dat=pd.read_excel(uniprot_filepath,header=0)
NSPfilePath='NetsurfP_Proteomes/Bovine_Mouse_proteome_complete.xlsx'
NSP_data=pd.read_excel(NSPfilePath)
###Bring in NP data and merge to get complete NP dataset###
NP_filepath='Input_data/NPs/NP_Database.xlsx'
NPUNdata=pd.read_excel(NP_filepath,header=0,sheet_name='NPUNID')
NPprop=pd.read_excel(NP_filepath,header=0,sheet_name='NP_Props')
NPdata=pd.merge(NPUNdata,NPprop,how="left",on='NPID')

In [43]:
#calculate Enrichment
#####MAYBE add binning here to to keep negative results and improve capapbilities######
MS_data_controls['Enrichment']= np.log2(MS_data_controls['Abundance']/MS_data_controls['Abundance_Controls'])
MS_data=MS_data_controls.drop(columns=['Abundance','Abundance_Controls'])
raw_prop_data=pd.merge(MS_data, uniprot_dat, how='left',on='Entry')

C:\Users\kmp95\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [44]:
MS_data_clean = raw_MS_data.copy()
PROT_cleaned_data = normalize_mass_length_1DF(raw_prop_data) #function found in data_prep_functions line 167, normalizes mass, length and mw by dividing all values by the max in the column
Protein_data_complete = pd.merge(PROT_cleaned_data, NSP_data, left_on='Entry', right_on='Entry') #merges netsurfp features and biopython features
#creates new column called asa_sum_normalized which is the asa_sum value divide by the mass of the protein
for df in [Protein_data_complete]:
    for col in ['asa_sum']:
        df[col+'_normalized'] = df[col] / df['Mass']

data_complete= pd.merge(Protein_data_complete,NPdata,how='left', on='NPUNID')
data_complete.drop(columns=['notes','Notes','BET','NPUNID'],inplace=True)
#Optional here to drop all enrichment values that are NA - also have to deal with positive/negative infinity values
# data_complete.dropna(subset=['Enrichment'],inplace=True)

In [56]:
data_complete.dropna(inplace=True)
data_complete= data_complete.replace([-np.inf],'-12')
data_complete=data_complete.replace([np.inf],'12')

In [57]:
labels=data_complete['Enrichment'].copy()
labels=np.ravel(labels)
df=data_complete.drop(['Entry','Sequence','Core Material','Ligand','NPID'],axis=1)

print(labels)
# print(labels)
# print(df)
first_frame = True #starting dataframe for saving metrics
correctness_frame = pd.DataFrame()
metrics_frame = pd.DataFrame()
print_metrics = 1 #0, doesn't show metrics while runnning for each model, 1 does show metrics
trials = 100


[-3.058068989758959 -2.6263436622455423 -2.6607046134587575 ...
 1.7890225357232328 '-12' '-12']


In [ ]:
labels

In [55]:
data_complete

,Entry,Enrichment,Sequence,Length,Mass,frac_aa_A,frac_aa_C,frac_aa_D,frac_aa_E,frac_aa_F,...,Core Material,Ligand,Dtem,Dh,Shaken,Centrifuged,NP_incubation Concentration (mg/mL),Incubation Concentration (mg/ml),Incubation Time (minutes),Temperature
0,P02769,-3.058069,MKWVTFISLLLLFSSAYSRGVFRRDTHKSEIAHRFKDLGEEHFKGL...,607.0,69293.0,0.079077,0.057661,0.065898,0.097199,0.049423,...,Iron Oxide,Carboxylate BSA,100.0,230.0,1.0,0.0,3.2,4.0,30.0,25.0
1,P02769,-2.626344,MKWVTFISLLLLFSSAYSRGVFRRDTHKSEIAHRFKDLGEEHFKGL...,607.0,69293.0,0.079077,0.057661,0.065898,0.097199,0.049423,...,Iron Oxide,Carboxylate BSA,100.0,230.0,1.0,0.0,3.2,4.0,30.0,25.0
2,P02769,-2.660705,MKWVTFISLLLLFSSAYSRGVFRRDTHKSEIAHRFKDLGEEHFKGL...,607.0,69293.0,0.079077,0.057661,0.065898,0.097199,0.049423,...,Iron Oxide,Carboxylate BSA,100.0,230.0,1.0,0.0,3.2,4.0,30.0,25.0
6,P41361,4.586080,MISNGIGTVTAGKRSICLLPLLLIGLWGCVTCHRSPVEDVCTAKPR...,465.0,52347.0,0.064516,0.019355,0.045161,0.075269,0.053763,...,Iron Oxide,Carboxylate BSA,100.0,230.0,1.0,0.0,3.2,4.0,30.0,25.0
7,P41361,4.631187,MISNGIGTVTAGKRSICLLPLLLIGLWGCVTCHRSPVEDVCTAKPR...,465.0,52347.0,0.064516,0.019355,0.045161,0.075269,0.053763,...,Iron Oxide,Carboxylate BSA,100.0,230.0,1.0,0.0,3.2,4.0,30.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3149,Q00896,-inf,MTPSISWGLLLLAGLCCLVPSFLAEDVQETDTSQKDQSPASHEIAT...,412.0,45823.0,0.070388,0.007282,0.050971,0.072816,0.060680,...,R101,none,800.0,680.0,1.0,1.0,125.0,0.2,60.0,25.0
3150,Q00896,-inf,MTPSISWGLLLLAGLCCLVPSFLAEDVQETDTSQKDQSPASHEIAT...,412.0,45823.0,0.070388,0.007282,0.050971,0.072816,0.060680,...,P25,none,378.0,410.0,1.0,1.0,62.5,0.2,60.0,25.0
3151,Q00896,1.789023,MTPSISWGLLLLAGLCCLVPSFLAEDVQETDTSQKDQSPASHEIAT...,412.0,45823.0,0.070388,0.007282,0.050971,0.072816,0.060680,...,P25,none,378.0,410.0,1.0,1.0,62.5,0.2,60.0,25.0
3152,Q00896,-inf,MTPSISWGLLLLAGLCCLVPSFLAEDVQETDTSQKDQSPASHEIAT...,412.0,45823.0,0.070388,0.007282,0.050971,0.072816,0.060680,...,P25,none,378.0,410.0,1.0,1.0,62.5,0.2,60.0,25.0


In [58]:

x_train, x_test, y_train, y_test = train_test_split(df,labels, test_size = 0.33, random_state=42)
# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=2016)
# for train_index, test_index in sss.split(df, labels):
# x_train = df.iloc[train_index]
# X_test = df.iloc[test_index]
# y_train = labels.iloc[train_index]
# y_test = labels.iloc[test_index]

rfg=RandomForestRegressor(n_estimators=100)
rfg.fit(x_train,y_train)
print(rfg.score(x_test,y_test))
print(rfg.feature_importances_)
# metrics_dict = {'AUC':metrics.roc_auc_score(y_test, rfg.predict_proba(x_test)[:, 1]),
#             'Accuracy':rfg.score(x_test, y_test), 'Recall':recall_score(y_test, rfg.predict(x_test)),
#             'Precision':precision_score(y_test, rfg.predict(x_test), zero_division=0), 'F1':f1_score(y_test, rfg.predict(x_test))}
# metrics_frame = pd.DataFrame.from_dict(data=metrics_dict,orient='index').transpose()

0.9999681602344153
[9.99921760e-01 2.22346775e-07 6.01045972e-07 3.28850935e-07
 5.96458240e-07 8.80757309e-07 4.17870146e-07 3.59552379e-07
 3.19444882e-07 2.78839481e-06 1.86980884e-06 2.17877683e-07
 3.40349131e-07 3.20147365e-06 1.06437586e-06 4.84770299e-07
 3.00224079e-07 6.97063210e-07 2.14266090e-06 3.10213969e-07
 3.40916047e-07 4.71621536e-07 3.60570788e-07 1.73777291e-06
 7.23113983e-07 8.64487110e-07 1.14721172e-06 1.29777998e-06
 1.19392275e-06 9.48903200e-07 1.22892577e-06 2.14824464e-07
 5.51107051e-07 3.21262628e-07 6.90644914e-07 8.71654786e-07
 2.05013630e-07 5.29798989e-07 1.01313169e-06 3.81778561e-07
 6.25352477e-07 6.41705450e-07 2.95448240e-07 2.86558745e-07
 2.37096736e-07 5.00695877e-07 3.47492655e-07 1.30598774e-07
 2.70355052e-07 1.89955741e-06 3.96283561e-07 1.40978729e-06
 5.68083607e-07 3.58533177e-07 5.48554481e-07 3.05013354e-07
 7.47399110e-07 3.19365925e-06 5.94403877e-07 9.31216532e-07
 4.66643966e-07 6.33956015e-07 1.45921486e-06 2.43577874e-07
 4.83